<a href="https://colab.research.google.com/github/JiYoon03/-Never-Miss-My-Blue-Bus-App/blob/main/AI%20Modeling%20Researcher%3AQ2_A_JiYoon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **의도 분류**

In [6]:
import random

# 1. 지식 제공 모듈
def knowledge_module(query):
    knowledge_responses = {
        "보험 상품": "회사 A의 보험 상품은 다양한 옵션이 있습니다. 예를 들어, 생명보험, 자동차보험, 건강보험 등이 있습니다.",
        "생명보험": "회사 A의 생명보험은 기본적인 사망 보장 외에도, 다양한 추가 혜택을 제공합니다.",
        "건강보험": "회사 A의 건강보험은 병원비와 치료비를 보장하며, 맞춤형 옵션을 선택할 수 있습니다.",
        "자동차 보험": "회사 A의 자동차 보험은 충돌 사고, 도난, 화재 등을 보장합니다. 여러 옵션 중에서 선택 가능합니다.",
        "가입 절차": "보험 가입 절차는 온라인 신청 후, 서류 제출 및 상담을 진행하는 방식입니다."
    }

    for key in knowledge_responses:
        if key in query:
            return knowledge_responses[key]

    return "저는 해당 정보는 잘 모르겠어요. 다른 질문을 해주실래요?"

# 2. ChitChat 모듈
def chit_chat_module(query):
    chit_chat_responses = {
        "날씨": "오늘 날씨는 맑고 기온은 약간 쌀쌀해요. 외출 시 옷을 따뜻하게 입는 게 좋겠어요.",
        "영화": "최근에 '위키드'가 정말 재미있었어요. 뮤지컬 영화 좋아하시나요?",
        "오늘 뭐해?": "일하고 있어요! 당신은 무엇을 하고 있나요?"
    }

    for key in chit_chat_responses:
        if key in query:
            return chit_chat_responses[key]

    return "그렇군요! 다른 얘기 해볼까요?"

# 3. 의도 분류 (간단한 키워드 기반)
def classify_intent(query):
    knowledge_keywords = ["보험", "가입", "상품", "자동차"]
    chit_chat_keywords = ["날씨", "영화", "오늘", "뭐해"]

    if any(keyword in query for keyword in knowledge_keywords):
        return "knowledge"
    elif any(keyword in query for keyword in chit_chat_keywords):
        return "chitchat"
    else:
        return "unknown"

# 4. 시스템 대화 흐름
def chatbot(query):
    intent = classify_intent(query)

    if intent == "knowledge":
        return knowledge_module(query)
    elif intent == "chitchat":
        return chit_chat_module(query)
    else:
        return "죄송합니다, 이해하지 못했어요. 다시 한 번 말씀해 주세요."

# 5. 장 대화
if __name__ == "__main__":
    print("안녕하세요! 무엇을 도와드릴까요?")

    while True:
        user_input = input("사용자: ")
        if user_input.lower() in ["exit", "종료", "quit"]:
            print("챗봇 종료!")
            break

        response = chatbot(user_input)
        print("챗봇: " + response)


안녕하세요! 무엇을 도와드릴까요?
사용자: A회사 보험 상품에 대해 알고 싶어
챗봇: 회사 A의 보험 상품은 다양한 옵션이 있습니다. 예를 들어, 생명보험, 자동차보험, 건강보험 등이 있습니다.
사용자: 영화 추천해줘
챗봇: 최근에 '위키드'가 정말 재미있었어요. 뮤지컬 영화 좋아하시나요?
사용자: 오늘 날씨에 대해 알려줘
챗봇: 오늘 날씨는 맑고 기온은 약간 쌀쌀해요. 외출 시 옷을 따뜻하게 입는 게 좋겠어요.
사용자: 생명보험에 대해 설명해줘
챗봇: 회사 A의 생명보험은 기본적인 사망 보장 외에도, 다양한 추가 혜택을 제공합니다.
사용자: 가입 절차는>
챗봇: 보험 가입 절차는 온라인 신청 후, 서류 제출 및 상담을 진행하는 방식입니다.
사용자: exit
챗봇 종료!


# **Dialogue State Tracking**

In [7]:
'''
  DialogueStateTracking
  사용자의 입력을 기반으로 대화 상태를 추적하며, 각 슬롯과 그 값을 관리합니다.
  update_state 메서드를 통해 입력에 맞춰 상태를 업데이트하고,
  generate_explanation 메서드에서 상태에 대한 설명을 생성합니다.
'''
class DialogueStateTracking:
    def __init__(self):
        self.slots = {}  # 슬롯 값을 추적할 dict
        self.history = []  # 대화 history

    def update_state(self, user_input):
        """
        사용자의 입력을 받아 대화 상태를 업데이트합니다.
        - 사용자 입력을 기반으로 슬롯 값을 추출하고 상태를 추적합니다.
        """
        if "보험" in user_input:
            self.slots["insurance"] = "requested"
        elif "자동차" in user_input and "보험" in user_input:
            self.slots["insurance_type"] = "자동차"
        elif "건강" in user_input and "보험" in user_input:
            self.slots["insurance_type"] = "건강"

        # 대화 history update
        self.history.append(user_input)

    def generate_explanation(self):
        """
        CoTE 모델을 단순화하여 상태 추적에 대한 설명을 생성합니다.
        """
        explanation = []
        for slot, value in self.slots.items():
            explanation.append(f"슬롯 '{slot}'의 값은 '{value}'로 추적되었습니다.")

        return explanation

    def get_current_state(self):
        return self.slots, self.history

'''
  KnowledgeModule
  사용자가 보험 관련 질문을 할 때 적절한 지식 기반 응답을 제공합니다.
  "보험 상품"에 대한 질문에 대해서는 회사의 보험 상품에 대한 정보를 반환합니다.
'''
class KnowledgeModule:
    def __init__(self):
        self.knowledge_responses = {
            "보험 상품": "회사의 다양한 보험 상품을 확인해보세요. 예를 들어, 생명보험, 자동차보험, 건강보험 등이 있습니다.",
            "생명보험": "생명보험은 고객의 생명과 관련된 위험에 대비하는 보험입니다.",
            "자동차 보험": "자동차 보험은 충돌, 도난, 화재 등의 위험에 대비할 수 있습니다.",
            "가입 절차": "가입 절차는 온라인 신청 후 서류 제출 및 상담을 진행합니다."
        }

    def get_response(self, query):
        """
        지식 기반의 응답을 제공합니다.
        """
        for key, response in self.knowledge_responses.items():
            if key in query:
                return response
        return "저는 해당 정보를 잘 모르겠어요. 다른 질문을 해주실래요?"
'''
  ChitChatModule
  자연스러운 대화를 처리하는 모듈로,
  사용자가 "날씨", "영화", "오늘 뭐해?"와 같은 질문을 하면 이에 맞는 응답을 생성합니다.
'''
class ChitChatModule:
    def __init__(self):
        self.chitchat_responses = {
            "날씨": "오늘 날씨는 맑고 기온은 약간 쌀쌀해요. 외출 시 따뜻한 옷을 입는 게 좋겠어요.",
            "영화": "최근에 '위키드'가 정말 재미있었어요. 액션 영화 좋아하시나요?",
            "오늘 뭐해?": "일하고 있어요! 당신은 무엇을 하고 있나요?"
        }

    def get_response(self, query):
        """
        chitchat 대화의 응답을 제공합니다.
        """
        for key, response in self.chitchat_responses.items():
            if key in query:
                return response
        return "그렇군요! 다른 얘기 해볼까요?"
'''
  Chatbot
  사용자의 입력을 받아 classify_intent 메서드로 의도를 분류하고,
  해당 의도에 맞는 모듈(KnowledgeModule 또는 ChitChatModule)을 활성화하여 응답을 생성합니다.
  또한, 대화 상태를 추적하고, CoTE 방식으로 설명을 생성하여 대화의 흐름을 명확히 합니다.
'''

class Chatbot:
    def __init__(self):
        self.dst = DialogueStateTracking()
        self.knowledge_module = KnowledgeModule()
        self.chitchat_module = ChitChatModule()

    def classify_intent(self, user_input):
        """
        사용자의 의도를 분류합니다.
        """
        knowledge_keywords = ["보험", "가입", "상품", "자동차", "건강"]
        chit_chat_keywords = ["날씨", "영화", "오늘", "뭐해"]

        if any(keyword in user_input for keyword in knowledge_keywords):
            return "knowledge"
        elif any(keyword in user_input for keyword in chit_chat_keywords):
            return "chitchat"
        else:
            return "unknown"

    def handle_user_input(self, user_input):
        """
        사용자의 입력을 처리하고, 대화 상태를 업데이트하며 응답을 생성합니다.
        """
        # 대화 상태 추적
        self.dst.update_state(user_input)

        # 의도 분류
        intent = self.classify_intent(user_input)

        if intent == "knowledge":
            # 지식 제공 모듈을 통해 응답 생성
            response = self.knowledge_module.get_response(user_input)
        elif intent == "chitchat":
            # 자연스러운 대화 모듈을 통해 응답 생성
            response = self.chitchat_module.get_response(user_input)
        else:
            response = "죄송합니다, 이해하지 못했어요. 다시 한 번 말씀해 주세요."

        # CoTE 기반 설명 생성
        explanation = self.dst.generate_explanation()

        # 현재 상태와 설명을 출력
        current_state, history = self.dst.get_current_state()
        print(f"현재 상태: {current_state}")
        print(f"대화 히스토리: {history}")
        print("설명: " + " | ".join(explanation))

        # 응답 반환
        return response


# 2. 예시 실행
if __name__ == "__main__":
    chatbot = Chatbot()

    print("안녕하세요! 무엇을 도와드릴까요?")

    while True:
        user_input = input("사용자: ")
        if user_input.lower() in ["exit", "종료", "quit"]:
            print("챗봇 종료!")
            break

        response = chatbot.handle_user_input(user_input)
        print("챗봇: " + response)


안녕하세요! 무엇을 도와드릴까요?
사용자: 자동차 보험에 대해 알려줘
현재 상태: {'insurance': 'requested'}
대화 히스토리: ['자동차 보험에 대해 알려줘']
설명: 슬롯 'insurance'의 값은 'requested'로 추적되었습니다.
챗봇: 자동차 보험은 충돌, 도난, 화재 등의 위험에 대비할 수 있습니다.
사용자: 영화 추천해줘
현재 상태: {'insurance': 'requested'}
대화 히스토리: ['자동차 보험에 대해 알려줘', '영화 추천해줘']
설명: 슬롯 'insurance'의 값은 'requested'로 추적되었습니다.
챗봇: 최근에 '위키드'가 정말 재미있었어요. 액션 영화 좋아하시나요?
사용자: 가입 절차를 알려줘
현재 상태: {'insurance': 'requested'}
대화 히스토리: ['자동차 보험에 대해 알려줘', '영화 추천해줘', '가입 절차를 알려줘']
설명: 슬롯 'insurance'의 값은 'requested'로 추적되었습니다.
챗봇: 가입 절차는 온라인 신청 후 서류 제출 및 상담을 진행합니다.
사용자: exit
챗봇 종료!
